In [1]:
# assignment 6
# MSDS 422
# Professor Anil Chaturvedi

# https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
# https://github.com/ageron/handson-ml/blob/master/12_distributed_tensorflow.ipynb

# more info
# https://www.datacamp.com/community/tutorials/cnn-tensorflow-python
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

# really wonderful resource for noobs
# https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

<div style="text-align: right"><b>pkg imports</b></div>

In [2]:
# basix
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import time

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#modeling
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import tensorflow as tf

# other

%matplotlib inline   
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<div style="text-align: right"><b>configs, table setup</b></div>

In [3]:
"""
for sci-kit:
‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

for tf:
idk
"""

#act_fun = "identity"
act_fun = "logistic"
#act_fun = "tanh"
#act_fun = "relu"

In [4]:
pd.options.display.float_format = "{:,.4f}".format
RANDOM_SEED = 8675309
responseVar = "label"

In [5]:
# setup for consolidated table, later

table = {"model 1":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 2":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 3":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 4":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 5":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 6":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 7":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 8":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None}
        }

pd.DataFrame(table).transpose().head(2)

,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,None,None,None,None,None
model 2,None,None,None,None,None


<div style="text-align: right"><b>defining things for model</b></div>

In [6]:
# activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# lambda x: 1 / (1 + np.exp(-x))

In [7]:
training_iters = 10
learning_rate = 0.01
b_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

<div style="text-align: right"><b>data setup for scikit</b></div>

In [8]:
mnist_train = pd.read_csv('train.csv')
mnist_test = pd.read_csv('test.csv')
#mnist = pd.concat([mnist_test, mnist_train.drop(columns = "label")]) # combines the rows of 2 datasets

In [9]:
print("le shape of le train is le : ", mnist_train.shape)
print("le shape of le test        : ", mnist_test.shape)

le shape of le train is le :  (42000, 785)
le shape of le test        :  (28000, 784)


In [10]:
# 'test' data is our future data
future_data = mnist_test
mnist = mnist_train
del(mnist_test)
del(mnist_train)

In [11]:
try:
    mnist_test
except NameError as e:
    print(e) # meow
    
try:
    mnist_train
except NameError as e:
    print(e) # meow

name 'mnist_test' is not defined
name 'mnist_train' is not defined


In [12]:
#X = mnist_scaled.loc[:, mnist_scaled.columns != responseVar]
#y = mnist_scaled[responseVar]
X = mnist.loc[:, mnist.columns != responseVar]
y = mnist[responseVar]

In [13]:
pixel744 = pd.DataFrame(mnist.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

In [14]:
scaler = StandardScaler()
np_mnist_x = scaler.fit_transform(X)
X_scaled = pd.DataFrame(np_mnist_x, index=X.index, columns=X.columns)

In [15]:
pd.DataFrame(X_scaled.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

,pixel774
count,"42,000.0000"
mean,0.0000
std,1.0000
min,-0.0347
max,40.2009


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED)

<div style="text-align: right"><b>data setup for tensorflow</b></div>

In [17]:
from keras.datasets import mnist
(tf_X_train, tf_y_train), (tf_X_test, tf_y_test) = mnist.load_data()

In [18]:
# scale ?

<div style="text-align: right"><b>model 1 : 2x1. 2 layers w 1 neuron each. sci-kit</b></div>

In [19]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter()

approximate start time :  Wed Oct 21 23:40:33 2020


In [20]:
twox1 = MLPClassifier(hidden_layer_sizes = (2, 1), # neurons comma layers
                      activation = act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(X_train,
                                               y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [21]:
stop1 = time.perf_counter()
tm1 = round(stop1 - start1)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm1)
print("time of execution in minutes : ", round(tm1/60, 1))

approximate completion time :  Wed Oct 21 23:41:20 2020
time of execution in seconds :  47
time of execution in minutes :  0.8


In [47]:
table["model 1"]["nbr_layers"] = "1"
table["model 1"]["nodes_p_layer"] = "2"
table["model 1"]["processing_time"] = tm1
table["model 1"]["testset_acc"] = twox1.score(X_test, y_test)
table["model 1"]["trainset_acc"] = twox1.score(X_train, y_train)

<div style="text-align: right"><b>model 2 : 2x2. 2 layers w 2 neurons each. I think. sci-kit</b></div>

In [23]:
print("approximate start time : ", time.asctime())
start2 = time.perf_counter()

approximate start time :  Wed Oct 21 23:41:20 2020


In [24]:
twox2 = MLPClassifier(hidden_layer_sizes = (2, 2), 
                      activation = act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(X_train,
                                               y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [25]:
stop2 = time.perf_counter()
tm2 = round(stop2 - start2)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm2)
print("time of execution in minutes : ", round(tm2/60, 1))

approximate completion time :  Wed Oct 21 23:42:07 2020
time of execution in seconds :  47
time of execution in minutes :  0.8


In [26]:
table["model 2"]["nbr_layers"] = "2"
table["model 2"]["nodes_p_layer"] = "2"
table["model 2"]["processing_time"] = tm2
table["model 2"]["testset_acc"] = twox2.score(X_test, y_test)
table["model 2"]["trainset_acc"] = twox2.score(X_train, y_train)

<div style="text-align: right"><b>model 3 : x1. 30 layers w 4 neurons each. sci-kit</b></div>

In [27]:
print("approximate start time : ", time.asctime())
start3 = time.perf_counter()

approximate start time :  Wed Oct 21 23:42:07 2020


In [43]:
thirtyx4 = MLPClassifier(hidden_layer_sizes = (30, 4), 
                      activation = act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(X_train,
                                               y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [29]:
stop3 = time.perf_counter()
tm3 = round(stop3 - start3)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm3)
print("time of execution in minutes : ", round(tm3/60, 1))

approximate completion time :  Wed Oct 21 23:43:44 2020
time of execution in seconds :  96
time of execution in minutes :  1.6


In [44]:
table["model 3"]["nbr_layers"] = "30"
table["model 3"]["nodes_p_layer"] = "4"
table["model 3"]["processing_time"] = tm3
table["model 3"]["testset_acc"] = thirtyx4.score(X_test, y_test)
table["model 3"]["trainset_acc"] = thirtyx4.score(X_train, y_train)

<div style="text-align: right"><b>model 4 : x1. 4 layers, each w 30 neurons. sci-kit</b></div>

In [31]:
print("approximate start time : ", time.asctime())
start4 = time.perf_counter()

approximate start time :  Wed Oct 21 23:43:44 2020


In [32]:
fourx30 = MLPClassifier(hidden_layer_sizes = (4, 30), 
                      activation = act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(X_train,
                                               y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [33]:
stop4 = time.perf_counter()
tm4 = round(stop4 - start4)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm4)
print("time of execution in minutes : ", round(tm4/60, 1))

approximate completion time :  Wed Oct 21 23:44:50 2020
time of execution in seconds :  66
time of execution in minutes :  1.1


In [45]:
table["model 4"]["nbr_layers"] = "4"
table["model 4"]["nodes_p_layer"] = "30"
table["model 4"]["processing_time"] = tm4
table["model 4"]["testset_acc"] = fourx30.score(X_test, y_test)
table["model 4"]["trainset_acc"] = fourx30.score(X_train, y_train)

<div style="text-align: right"><b>model 5 : 2x1. 2 layers w 1 neurons each</b></div>

In [35]:
print("approximate start time : ", time.asctime())
start5 = time.perf_counter()

approximate start time :  Wed Oct 21 23:44:50 2020


In [36]:
# tf model

In [37]:
stop5 = time.perf_counter()
tm5 = round(stop5 - start5)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm5)
print("time of execution in minutes : ", round(tm5/60, 1))

approximate completion time :  Wed Oct 21 23:44:50 2020
time of execution in seconds :  0
time of execution in minutes :  0.0


In [38]:
table["model 5"]["nbr_layers"] = "2"
table["model 5"]["nodes_p_layer"] = "1"
table["model 5"]["processing_time"] = tm5
#table["model 5"]["testset_acc"] = #tf test score
#table["model 5"]["trainset_acc"] = #tf train score

<div style="text-align: right"><b>model 6 : 2x2. 2 layers w 2 neurons each</b></div>

In [39]:
print("approximate start time : ", time.asctime())
start6 = time.perf_counter()

approximate start time :  Wed Oct 21 23:44:50 2020


<div style="text-align: right"><b>model 7 : 30x4. 30 layers w 4 neurons each</b></div>

In [40]:
print("approximate start time : ", time.asctime())
start7 = time.perf_counter()

approximate start time :  Wed Oct 21 23:44:50 2020


<div style="text-align: right"><b>model 8 : 4x30. 4 layers w 30 neurons each</b></div>

In [41]:
print("approximate start time : ", time.asctime())
start8 = time.perf_counter()

approximate start time :  Wed Oct 21 23:44:50 2020


In [48]:
print("processing time in seconds : ")
pd.DataFrame(table).transpose()

processing time in seconds : 


,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,1,2,47,0.3324,0.3213
model 2,2,2,47,0.5489,0.5337
model 3,30,4,96,0.9973,0.8948
model 4,4,30,66,0.8926,0.8607
model 5,2,1,0,None,None
model 6,None,None,None,None,None
model 7,None,None,None,None,None
model 8,None,None,None,None,None
